## 02 - The True Power of Qurry

In the previous section, we have learned how to use the basic function of Qurry. In this section, we will learn that Qurry provides a more advanced approach, where the true power of Qurry lies.


In [1]:
from qurry import EntropyMeasure, BackendManager
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from pathlib import Path

experiment_executor_02 = EntropyMeasure()

In [2]:
from qiskit.providers.basic_provider import BasicProvider

basic_provider = BasicProvider()
backend_sim = basic_provider.get_backend("basic_simulator")

## 2.1 - Launching a multiJob

Consider a scenario where you have multiple circuits that you want to run on a backend at the same time, you can use `multiOutput` to launch a multiJob.

For the example below, we will show how to submit 100 circuits with 100 sets of randomized unitaries and 4096 shots, and mesure the entropy with multiple subsystem divisions at the same time.


### 1. Loading 10 circuits

(Topological Paramagnetic State is already the most complicated case in Qurry, so we will use it as our example. Ususally I prefer some more complicated circuits like a spin chain model with 20+ trotter steps, but it's not necessary for this example.)


In [3]:
from qurry.recipe import TopologicalParamagnet
sample = TopologicalParamagnet(8, 'period')
print(sample)

     ┌───┐         
q_0: ┤ H ├─■─────■─
     ├───┤ │     │ 
q_1: ┤ H ├─■──■──┼─
     ├───┤    │  │ 
q_2: ┤ H ├─■──■──┼─
     ├───┤ │     │ 
q_3: ┤ H ├─■──■──┼─
     ├───┤    │  │ 
q_4: ┤ H ├─■──■──┼─
     ├───┤ │     │ 
q_5: ┤ H ├─■──■──┼─
     ├───┤    │  │ 
q_6: ┤ H ├─■──■──┼─
     ├───┤ │     │ 
q_7: ┤ H ├─■─────■─
     └───┘         


Loading circuits to `Qurry`


In [4]:
for i in range(100):
    experiment_executor_02.add(sample)


Check the name or serial number of the circuit in `Qurry` by `experiment_executor_02.waves.keys()`


In [5]:
print(experiment_executor_02.waves.keys())

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])


### 2. Preparing configurations

To perform randomized measurement, multiple randomized unitary configurations required. We use a list to store all configurations.

For randomized measurement, each configuration is in this form of dictionary:

```python
{
    'waves': 'the circuit to measure',
    'tags': 'tags for this job',
    'times': 100,
}
```


In [6]:
from typing import TypedDict, Hashable, Iterable, Union, Optional

class randomizedConfig(TypedDict):
    wave: Hashable
    """The name or serial number of circuit in Qurry."""
    tags: Hashable
    """You can metion tags to filter experiment."""
    times: int
    """Default: 100 in :cls:`RandomizedMeasure`"""
    

In [7]:
config_list = [{
    'wave': i,
    'tags': ('topParamagnet', int(i/10)),
    'times': 100,
} for i in range(10)]

### 3. Launching multiJob

Attenetion, this example may make your computer go brrrr.


In [8]:
# please un-comment this cell to run it.

hash_id = experiment_executor_02.multiOutput(
    config_list=config_list,
    backend=backend_sim,
    save_location=Path('./'),
    summoner_name='example.multiOutput',
    shots=4096,
)

# It takes 10m 41.2s to run this cell on my computer.

| MultiManager building...
| Write "example.multiOutput.qurry.001", at location "example.multiOutput.qurry.001"


| 0/10   0%|          | -  - 00:00 < ?

| Export multimanager...


| 0/10 - exporting - 00:00 < ?

| No quantity to export.
| Export multi.config.json for 205f52f8-031a-4a09-b622-b187ccc2efa7


...

| 0/10   0%|          | -  - 00:00 < ?

| Export multimanager...


| 0/10 - exporting - 00:00 < ?

| No quantity to export.
| Export multi.config.json for 205f52f8-031a-4a09-b622-b187ccc2efa7


| 0/10 - Exporting and writring... - 00:00 < ?

The `multiOutput` will return an ID of this multiJob,

you can use it to check the status of this multiJob in `experiment_executor_02.multimanagers[(the ID returned)]`.


In [9]:
hash_id
# The hashID of this multiOutput

'205f52f8-031a-4a09-b622-b187ccc2efa7'

In [10]:
experiment_executor_02.multimanagers

MultiManagerContainer(num=4, {
  '205f52f8-031a-4a09-b622-b187ccc2efa7': '<MultiManager(id=205f52f8-031a-4a09-b622-b187ccc2efa7,
  name=example.multiOutput.qurry.001,
  jobstype=local,
  ...)>'})

### 3. The automatic export of multiJob

After the multiJob is finished, you may notice that there are some new files in your folder, they are the results of your multiJob.

Qurry will automatically export the results of multiJob to your folder, and you can use `multiRead` to load them. We will show how to use `multiRead` in later section.


### 4. Make multiple analysis with multiple subsystem divisions

Attenetion, this example may make your computer go brrrr, too.
The post-processing calculation already boosts by multiprocesses, cython and Rust, speeding up the calculation. In early version, such calculation can take all day, even few days to finish.


In [11]:
subsytems = [
    2,
    4,
    6,
    (2, 4),
    (4, 6),
    (2, 6),
    (-2, 2),
    (-4, 2),
    (-2, 4),
    3,
    (5, 7),
    (4, 7),
]
subsytems = [
    list(range(i)) if isinstance(i, int) else [j % 8 for j in range(*i)]
    for i in subsytems
]
# this is a list of subsystems that we want to measure
subsytems

[[0, 1],
 [0, 1, 2, 3],
 [0, 1, 2, 3, 4, 5],
 [2, 3],
 [4, 5],
 [2, 3, 4, 5],
 [6, 7, 0, 1],
 [4, 5, 6, 7, 0, 1],
 [6, 7, 0, 1, 2, 3],
 [0, 1, 2],
 [5, 6],
 [4, 5, 6]]

In [14]:
for qs in subsytems:
    name = "bits_" + "".join([str(qi) for qi in qs])
    print("| Name:", name)
    
    # please decoment this cell to run it.
    
    experiment_executor_02.multiAnalysis(
        summoner_id=hash_id,
        selected_qubits=qs,
        analysis_name=name,
        no_serialize=True,
    )
    
# 38m 15.1s to run this cell on my computer.

| Name: bits_01


| 0/10 - Analysis:  - 00:00 < ?

| "bits_01" has been completed.
| Export multimanager...


| 0/10 - exporting - 00:00 < ?

| 0/1 - exporting quantity - 00:00 < ?

...

| "bits_456" has been completed.
| Export multimanager...


| 0/10 - exporting - 00:00 < ?

| 0/12 - exporting quantity - 00:00 < ?

| Export multi.config.json for 205f52f8-031a-4a09-b622-b187ccc2efa7


| 0/10 - Exporting and writring... - 00:00 < ?

## 2.2 Export and Compress

In this section, we will show how to use `multiWrite` to export the results of multiJob to a folder or export as a tar.xz file when `compress=True`.


In [16]:
experiment_executor_02.multiWrite(
    save_location=Path('./'),
    summoner_id=hash_id,
    compress=True,
)

| Export multimanager...


| 0/10 - exporting - 00:00 < ?

| 0/12 - exporting quantity - 00:00 < ?

| Export multi.config.json for 205f52f8-031a-4a09-b622-b187ccc2efa7


| 0/10 - Exporting and writring... - 00:00 < ?

| Compress multimanager of 'example.multiOutput.qurry.001'...done


'205f52f8-031a-4a09-b622-b187ccc2efa7'

## 2.3 Reading and Writing Data

In this section, we will show how to use `multiRead` to load the results of multiJob.


In [17]:
hashID = experiment_executor_02.multiRead(
    save_location=Path('./'),
    summoner_name='example.multiOutput.qurry.001',
)

| Retrieve example.multiOutput.qurry.001...
| at: example.multiOutput.qurry.001
| Found the tarfile 'example.multiOutput.qurry.001.qurry.tar.xz' in '.', decompressing is available.
The following files '['example.multiOutput.qurry.001/bits_0123.quantity.json']' are fitting giving 'taglist_name' and 'name', choosing the 'example.multiOutput.qurry.001/bits_0123.quantity.json'.
The following files '['example.multiOutput.qurry.001/bits_0123.quantity.json']' are fitting giving 'taglist_name' and 'name', choosing the 'example.multiOutput.qurry.001/bits_0123.quantity.json'.
The following files '['example.multiOutput.qurry.001/bits_23.quantity.json']' are fitting giving 'taglist_name' and 'name', choosing the 'example.multiOutput.qurry.001/bits_23.quantity.json'.
The following files '['example.multiOutput.qurry.001/bits_456701.quantity.json']' are fitting giving 'taglist_name' and 'name', choosing the 'example.multiOutput.qurry.001/bits_456701.quantity.json'.
The following files '['example.mult

| 0/10   0%|          | - 10 experiments found, loading by 16 workers. - 00:00 < ?

---

## Post-Process Availablities and Version Info


In [18]:
from qurry import __version__
from qurry.process import AVAIBILITY_STATESHEET
print("| Qurry version:", __version__)
print(AVAIBILITY_STATESHEET)

| Qurry version: 0.9.2.dev1
 | Qurry version: 0.9.2.dev1
--------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ................... Python Cython Rust  
 - randomized_measure
   - entangled_entropy.entropy_core_2 ....... Yes    Depr.  Yes   
   - entangle_entropy.purity_cell_2 ......... Yes    Depr.  Yes   
   - wavefunction_overlap ................... Yes    Depr.  Yes   
   - echo_cell .............................. Yes    Depr.  Error 
 - utils
   - randomized ............................. Yes    Depr.  Yes   
   - construct .............................. Yes    No     Yes   
   - dummy .................................. Yes    No     Yes   
 - hadamard_test
   - purity_echo_core ....................... Yes    No     Yes   
 - magnet_square
   - magnsq_core ............................ Yes    No     No    
--------------------------------------------------------
   + Yes ...... Working normally.
   + Error .... Exception occurre